In [1]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    project_id="2720f785-a3a2-4472-969d-19a53891c817"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Register circuit

In [2]:
# species = client.search_entity(
#     entity_type=models.Species, query={"name__ilike": "Homo sapiens"}
# ).one()

# subject = models.Subject(
#     name="my-subject", description="my-subject-description", sex="male", species=species
# )
# subject = client.register_entity(subject)

# brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


# circuit = models.Circuit(
#     name="my-circuit",
#     description="my-circuit",
#     subject=subject,
#     brain_region=brain_region,
#     number_synapses=2,
#     number_neurons=5,
#     number_connections=10,
#     scale="microcircuit",
#     build_category="em_reconstruction",
# )

# circuit = client.register_entity(circuit)

# print(circuit)
# # 5b2368ce-f632-47a5-9d5d-3c1c4159a232

# Create SimulationForm using Cirucit

In [3]:
import obi_one as obi

# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(name="O1 Simulation", description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str="5b2368ce-f632-47a5-9d5d-3c1c4159a232"), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScan

In [4]:
grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate', data_postprocessing_method='save', db_client=client)

[2025-06-18 12:42:09,598] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-18 12:42:09,598] INFO: No multiple value parameters found.
[2025-06-18 12:42:09,598] INFO: 
COORDINATE PARAMETERS
[2025-06-18 12:42:09,599] INFO: No coordinate parameters.
[2025-06-18 12:42:09,599] INFO: <entitysdk.client.Client object at 0x119da3b30>
[2025-06-18 12:42:10,195] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation-campaign "HTTP/1.1 200 OK"
initialize.circuit is a CircuitFromID instance.
Saving simulation to database...
[2025-06-18 12:42:11,224] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation "HTTP/1.1 200 OK"
[2025-06-18 12:42:11,229] INFO: create_bbp_workflow_campaign_config() not yet complete.


ValidationError: 1 validation error for SimulationGeneration
generated.0
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=[Simulation(assets=[], id...ameters={'foo': 'bar'})], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_attributes_type

# Register SimulationCampaign
Notice how the circuit is associated

In [ ]:
# campaign = client.register_entity(
#     models.SimulationCampaign(
#         name="my-campaign",
#         description="my-campaign-description",
#         entity_id=circuit.id,
#         scan_parameters={"foo": "bar"},
#     )
# )

# print(campaign)
# # a2aac501-1b42-4bf5-a0f5-7826279c0276

# Register 2 Simulations
Notice how the circuit and campaign are associated

In [ ]:
# simulations = [
#     client.register_entity(
#         models.Simulation(
#             name=f"sim-{i}",
#             description=f"sim-{i}",
#             scan_parameters={"foo": "bar"},
#             entity_id=circuit.id,
#             simulation_campaign_id=campaign.id,
#         )
#     )
#     for i in range(2)
# ]

# print(simulations[0])
# # 7fef488a-0fcc-4ca4-980d-1de2bb76eeb

# print(simulations[1])
# # 51a0a1ba-333a-45ef-9c22-56f538ac8729

# Register 2 Simulation Results

In [ ]:
# simulation_results = [
#     client.register_entity(models.SimulationResult(name=f"result-{i}", description=f"result-{i}"))
#     for i in range(2)
# ]